In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import gc

pd.options.display.max_columns = 999

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import json
from collections import Counter
import time
from pathlib import Path
import os
from tqdm import tqdm
import re
from sklearn.metrics import f1_score

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [3]:
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [4]:
BASE = Path.cwd()
DATA =  BASE / 'data'

RAW_SPLITS =  DATA / 'raw_splits_year'
PREC_SPLITS =  DATA / 'prec_splits_year'

TRAIN = DATA / 'data_train.csv.zip'
TEST = DATA / 'data_test.csv.zip'

In [6]:
TRAIN = DATA / 'train_v2.csv'
TEST = DATA / 'test_v2.csv'

split_train = pd.read_csv('data_train.csv', encoding='latin-1')
split_test = pd.read_csv('data_test.csv', encoding='latin-1')


In [7]:
split_train['Morosidad'] = np.max(split_train[['SITUACION_DEUDA_BOLETA_2',
 'SITUACION_DEUDA_BOLETA_3',
 'SITUACION_DEUDA_BOLETA_4',
 'SITUACION_DEUDA_BOLETA_5',
 'SITUACION_DEUDA_BOLETA_6',
 'SITUACION_DEUDA_BOLETA_7',
 'SITUACION_DEUDA_BOLETA_8',
 'SITUACION_DEUDA_BOLETA_9',
 'SITUACION_DEUDA_BOLETA_10']].values, axis=1)
split_train.shape

(256215, 127)

In [8]:
split_train = split_train.drop(['SITUACION_DEUDA_BOLETA_1', 
 'SITUACION_DEUDA_BOLETA_2',
 'SITUACION_DEUDA_BOLETA_3',
 'SITUACION_DEUDA_BOLETA_4',
 'SITUACION_DEUDA_BOLETA_5',
 'SITUACION_DEUDA_BOLETA_6',
 'SITUACION_DEUDA_BOLETA_7',
 'SITUACION_DEUDA_BOLETA_8',
 'SITUACION_DEUDA_BOLETA_9',
 'SITUACION_DEUDA_BOLETA_10'], axis=1)
split_train.shape

(256215, 117)

In [9]:
def convert_nan(val):
    if (val <=0):
        return np.nan
    else:
        return val

In [10]:
split_train['EDAD_ANIO_ACTUAL'] = split_train['EDAD_ANIO_ACTUAL'].apply(convert_nan)
split_train['EDAD_ANIO_ESTUDIO'] = split_train['EDAD_ANIO_ESTUDIO'].apply(convert_nan)

split_test['EDAD_ANIO_ACTUAL'] = split_test['EDAD_ANIO_ACTUAL'].apply(convert_nan)
split_test['EDAD_ANIO_ESTUDIO'] = split_test['EDAD_ANIO_ESTUDIO'].apply(convert_nan)


In [11]:
split_train['TIPO_BECA'] = split_train['TIPO_BECA'].fillna('TIPO_BECA')
split_test['TIPO_BECA'] = split_test['TIPO_BECA'].fillna('TIPO_BECA')


In [12]:
split_train['TRABAJA'] = split_train['TRABAJA'].fillna(0)
split_test['TRABAJA'] = split_test['TRABAJA'].fillna(0)


In [13]:
split_train['INGRESO_PROPIO'] = split_train['INGRESO_PROPIO'].fillna(0)
split_test['INGRESO_PROPIO'] = split_test['INGRESO_PROPIO'].fillna(0)


In [14]:
split_train['NRO_HERMANOS'] = split_train['NRO_HERMANOS'].fillna(0)
split_test['NRO_HERMANOS'] = split_test['NRO_HERMANOS'].fillna(0)


In [15]:
split_train['TOT_CURSOS'] = split_train['CURSOSAPROBADOS']+split_train['CURSOSDESAPROBADOS']+split_train['CURSOSNSP']
split_test['TOT_CURSOS'] = split_test['CURSOSAPROBADOS']+split_test['CURSOSDESAPROBADOS']+split_test['CURSOSNSP']


In [16]:
split_train['TOT_CRED'] = split_train['CREDAPROBADOS']+split_train['CREDDESAPROBADOS']+split_train['CURSOSNSP']
split_test['TOT_CRED'] = split_test['CREDAPROBADOS']+split_test['CREDDESAPROBADOS']+split_test['CURSOSNSP']


In [17]:
split_train['CARRERACONCAT'] = split_train['CARRERA'] + split_train['CARRERA_ACTUAL']
split_train['CARRERADISTRITO'] = split_train['CARRERA'] + split_train['CDIST_DESCRIPCION']

split_test['CARRERACONCAT'] = split_test['CARRERA'] + split_test['CARRERA_ACTUAL']
split_test['CARRERADISTRITO'] = split_test['CARRERA'] + split_test['CDIST_DESCRIPCION']


In [18]:
    from category_encoders import *
    X_train = split_train.drop(['Morosidad'], axis=1)
    y_train = split_train.Morosidad
    
    # use target encoding to encode two categorical features
    enc = TargetEncoder(cols=['CARRERA', 'CARRERA_ACTUAL', 'CARRERADISTRITO', 'CARRERACONCAT']).fit(X_train, y_train)

    # transform the datasets
    train = enc.transform(X_train)
    split_test = enc.transform(split_test)
    split_train = pd.concat((train, y_train), axis=1)
    

In [19]:
    X_train = split_train.drop(['Morosidad'], axis=1)
    y_train = split_train.Morosidad
    
    # use target encoding to encode two categorical features
    enc = TargetEncoder(cols=[
       'CDIST_DESCRIPCION']).fit(X_train, y_train)

    # transform the datasets
    train = enc.transform(X_train)
    split_test = enc.transform(split_test)
    split_train = pd.concat((train, y_train), axis=1)

In [20]:
    ## Parsear columnas

    split_train['PORC_ASISTENCIA'] = split_train['PORC_ASISTENCIA'].replace('SIN ASISTENCIA TOMADA', np.nan)
    split_train['PORC_ASISTENCIA'] = split_train['PORC_ASISTENCIA'].astype(float)

    split_train['SITUACION_EGRESO'] = split_train['SITUACION_EGRESO'].replace('NO EGRESO', 0) 
    split_train['SITUACION_EGRESO'] = split_train['SITUACION_EGRESO'].replace('EGRESO', 1) 
    split_train['SITUACION_EGRESO'] = split_train['SITUACION_EGRESO'].astype(int)

    split_train['SEXO'] = split_train['SEXO'].replace('M', 0) 
    split_train['SEXO'] = split_train['SEXO'].replace('F', 1) 
    split_train['SEXO'] = split_train['SEXO'].astype(int)

    split_test['PORC_ASISTENCIA'] = split_test['PORC_ASISTENCIA'].replace('SIN ASISTENCIA TOMADA', np.nan)
    split_test['PORC_ASISTENCIA'] = split_test['PORC_ASISTENCIA'].astype(float)

    split_test['SITUACION_EGRESO'] = split_test['SITUACION_EGRESO'].replace('NO EGRESO', 0) 
    split_test['SITUACION_EGRESO'] = split_test['SITUACION_EGRESO'].replace('EGRESO', 1) 
    split_test['SITUACION_EGRESO'] = split_test['SITUACION_EGRESO'].astype(int)

    split_test['SEXO'] = split_test['SEXO'].replace('M', 0) 
    split_test['SEXO'] = split_test['SEXO'].replace('F', 1) 
    split_test['SEXO'] = split_test['SEXO'].astype(int)

In [21]:
    ## Remover las columnas

    split_train = split_train.drop(['VIVIENDA_DONDE_VIVE','CDEPA_DESCRIPCION', 'CPROV_DESCRIPCION',], axis=1, errors='coerce')
    split_test = split_test.drop(['VIVIENDA_DONDE_VIVE','CDEPA_DESCRIPCION', 'CPROV_DESCRIPCION',], axis=1, errors='coerce')


In [22]:
    ## Remover las columnas

    split_train = split_train.drop(['TIPO_SEMESTRE'], axis=1)
    split_test = split_test.drop(['TIPO_SEMESTRE'], axis=1)


In [23]:
    ## Aplicando one hot encoding

    X_train = split_train.drop(['Morosidad'], axis=1)
    y_train = split_train.Morosidad

    enc = OneHotEncoder(cols=['TIPO_VIVIENDA_DONDE_VIVE', 'VIVE_CON', 'TIPO_COLEGIO', 'ESTADO_CIVIL']).fit(X_train)

    train = enc.transform(X_train)
    split_test = enc.transform(split_test)
    split_train = pd.concat((train, y_train), axis=1)


In [24]:
    ## Sumariazacion de columnas inutiles

    filter_col_a = [col for col in split_train.columns if col.startswith('CURSO_MAT_CICLO_')]
    split_train['CURSO_MAT_CICLO_SUM'] = split_train[filter_col_a].sum(axis=1)
    split_test['CURSO_MAT_CICLO_SUM'] = split_test[filter_col_a].sum(axis=1)

    filter_col_b = [col for col in split_train.columns if col.startswith('CRED_MAT_CICLO_')]
    split_train['CRED_MAT_CICLO_SUM'] = split_train[filter_col_b].sum(axis=1)
    split_test['CRED_MAT_CICLO_SUM'] = split_test[filter_col_b].sum(axis=1)
    
    split_train['MAT_CICLO_PON'] = 0
    split_test['MAT_CICLO_PON'] = 0
    for a, b in zip(filter_col_a, filter_col_b):
        split_train['MAT_CICLO_PON'] += split_train[a] * split_train[b]
        split_test['MAT_CICLO_PON'] += split_test[a] * split_test[b]
        
    split_train = split_train.drop(filter_col_b, axis=1)
    split_test = split_test.drop(filter_col_b, axis=1)
    split_train = split_train.drop(filter_col_a, axis=1)
    split_test = split_test.drop(filter_col_a, axis=1)

In [25]:
    filter_col_a = [col for col in split_train.columns if col.startswith('CURSO_MAT_DES_CICLO_')]
    split_train['CURSO_MAT_DES_CICLO_SUM'] = split_train[filter_col_a].sum(axis=1)
    split_test['CURSO_MAT_DES_CICLO_SUM'] = split_test[filter_col_a].sum(axis=1)

    filter_col_b = [col for col in split_train.columns if col.startswith('CRED_MAT_DES_CICLO_')]
    split_train['CRED_MAT_DES_CICLO_SUM'] = split_train[filter_col_b].sum(axis=1)
    split_test['CRED_MAT_DES_CICLO_SUM'] = split_test[filter_col_b].sum(axis=1)
    
    split_train['MAT_DES_CICLO_PON'] = 0
    split_test['MAT_DES_CICLO_PON'] = 0
    for a, b in zip(filter_col_a, filter_col_b):
        split_train['MAT_DES_CICLO_PON'] += split_train[a] * split_train[b]
        split_test['MAT_DES_CICLO_PON'] += split_test[a] * split_test[b]
        
    split_train = split_train.drop(filter_col_b, axis=1)
    split_test = split_test.drop(filter_col_b, axis=1)
    split_train = split_train.drop(filter_col_a, axis=1)
    split_test = split_test.drop(filter_col_a, axis=1)

In [26]:
    split_train['SEM_NOTAS_HIST_INICIA_CARRERA'] = split_train['SEM_NOTAS_HIST_INICIA_CARRERA'].replace('NINGU', np.nan)
    split_train['SEM_NOTAS_HIST_INICIA_CARRERA'] = split_train['SEM_NOTAS_HIST_INICIA_CARRERA'].astype(float)
    split_test['SEM_NOTAS_HIST_INICIA_CARRERA'] = split_test['SEM_NOTAS_HIST_INICIA_CARRERA'].replace('NINGU', np.nan)
    split_test['SEM_NOTAS_HIST_INICIA_CARRERA'] = split_test['SEM_NOTAS_HIST_INICIA_CARRERA'].astype(float)

    split_train['SEM_NOTAS_HIST_ULTIMO_CARRERA'] = split_train['SEM_NOTAS_HIST_ULTIMO_CARRERA'].replace('NINGU', np.nan)
    split_train['SEM_NOTAS_HIST_ULTIMO_CARRERA'] = split_train['SEM_NOTAS_HIST_ULTIMO_CARRERA'].astype(float)
    split_test['SEM_NOTAS_HIST_ULTIMO_CARRERA'] = split_test['SEM_NOTAS_HIST_ULTIMO_CARRERA'].replace('NINGU', np.nan)
    split_test['SEM_NOTAS_HIST_ULTIMO_CARRERA'] = split_test['SEM_NOTAS_HIST_ULTIMO_CARRERA'].astype(float)
    
    split_train['TOT_CIC'] = split_train['SEM_NOTAS_HIST_ULTIMO_CARRERA'] - split_train['SEM_NOTAS_HIST_INICIA_CARRERA'] 
    split_test['TOT_CIC'] = split_test['SEM_NOTAS_HIST_ULTIMO_CARRERA'] - split_test['SEM_NOTAS_HIST_INICIA_CARRERA']
    
    split_train.to_csv('train_k_v4.csv', index=False)
    split_test.to_csv('test_k_v4.csv', index=False)


In [28]:
train = pd.read_csv('train_k_v4.csv', encoding='latin-1')
test = pd.read_csv('test_k_v4.csv', encoding='latin-1')
train.shape, test.shape

((256215, 87), (44072, 86))

In [29]:
from sklearn.model_selection import StratifiedKFold
import lightgbm

In [30]:
#Preparando el dataset
X_train, y_train = train.drop(['COD_ALUMNO', 'Morosidad'], axis=1), train['Morosidad']
print(X_train.shape, y_train.shape)
X_test = test.drop(['COD_ALUMNO'], axis=1)
print(X_test.shape)
#Folds
skfolds = StratifiedKFold(n_splits=6, random_state=42, shuffle=True)

(256215, 85) (256215,)
(44072, 85)


In [31]:
from sklearn.metrics import f1_score

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True

In [32]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, f1_score
from scipy.stats import mode

def print_results(trn_aucs, val_aucs):
    train_auc, train_intconf = np.mean(trn_aucs), 2 * np.std(trn_aucs)
    val_auc, val_intconf = np.mean(val_aucs), 2 * np.std(val_aucs)
    
    train_gini, val_gini = (train_auc - 0.5) * 2, (val_auc - 0.5) * 2
    train_gini_intconf, val_gini_intconf = train_intconf * 2, val_intconf * 2
    
    print(f'Train AUC: {100*train_auc:.2f} +/- {100*train_intconf:.2f} | '
          f'Val AUC: {100*val_auc:.2f} +/- {100*val_intconf:.2f} | '
          f'Train Gini: {100*train_gini:.2f} +/- {100*train_gini_intconf:.2f} | '
          f'Val Gini: {100*val_gini:.2f} +/- {100*val_gini_intconf:.2f}')  
    return np.mean(trn_aucs), np.mean(val_aucs)

def test_lgbm(lgbm, X, y, X_test, kfolds, cat_features, params, num_rounds=1000):
    trn_aucs, val_aucs, trn_f1, val_f1 = [], [], [], []
    y_pred = np.zeros(len(X))
    y_test = np.zeros(len(X_test))
    models = []
    evals_result = []
    for trn_idx, val_idx in kfolds.split(X, y):
        eval_result = {}
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
        X_trn, y_trn = X.iloc[trn_idx], y.iloc[trn_idx]
        dtrn = lgbm.Dataset(X_trn, y_trn)
        dval = lgbm.Dataset(X_val, y_val)
        bst = lgbm.train(params, dtrn, num_rounds, [dval],
                  early_stopping_rounds=30, categorical_feature=cat_features, feval=lgb_f1_score, 
                         evals_result=eval_result,
                  verbose_eval=False)
        evals_result.append(eval_result)
        y_trn_pred = bst.predict(X_trn)
        y_val_pred = bst.predict(X_val)
        trn_aucs.append(roc_auc_score(y_trn, y_trn_pred))
        val_aucs.append(roc_auc_score(y_val, y_val_pred))
        
        print(f'No. estimators: {bst.best_iteration} | '
              f'Train AUC: {100*trn_aucs[-1]:.2f} | '
              f'Val AUC: {100*val_aucs[-1]:.2f} | '
              f'Train Gini: {(100*trn_aucs[-1]-50)*2:.2f} | '
              f'Val Gini: {(100*val_aucs[-1]-50)*2:.2f} | ')
        
        y_tst_pred = bst.predict(X_test)
        y_test += y_tst_pred
        y_pred[val_idx] = y_val_pred
        
        models.append(bst)
        
    print()
    print_results(trn_aucs, val_aucs)
    print()
    return y_test / kfolds.n_splits, y_pred, models, evals_result

In [37]:
X_train['TIPO_BECA'].value_counts()

SIN BECA                                154995
CONVENIO POLICIA NACIONAL                37308
DSCTO. PADRES CON VARIOS HIJOS           16246
BECA RENDIMIENTO Y STCION. ECONOMICA     12194
CONVENIO EJERCITO PERUANO                 9228
CONVENIO FUERZA AEREA DEL PERU            7529
DSCTO. HIJOS DE GRADUADOS DE URP          7239
CONVENIO MARINA DE GUERRA DEL PERU        5475
HIJO DE TRABAJADOR ADMINISTRATIVO         2487
DEPORTITAS CALIFICADOS                    1039
HIJO DE DOCENTE TIEMPO PARCIAL             955
BECA ORFANDAD                              876
HIJO DE DOCENTE TIEMPO COMPLETO            384
BECA PRIMEROS PUESTOS REGULAR              129
TRABAJADOR ADMINISTRATIVO                  110
CONVENIO HOSP.DA.CARRION                     7
TIPO_BECA                                    7
BECA_HIJO TRAB_JUBILADO                      7
Name: TIPO_BECA, dtype: int64

In [33]:
%%time
#### SUBMISSION ####
params = {'bagging_fraction': '0.9761679761799377',
  'feature_fraction': '0.7816500852961176',
  'lambda_l1': '0.13407390174373368',
  'lambda_l2': '2.212572925712077',
  'max_depth': '8',
  'min_child_weight': '5.232240193423142',
  'min_split_gain': '0.09208559917489269',
  'num_leaves': '44',
 'objective':'binary', 'metric': 'None', 'gpu_device_id': '1'}
y_test, y_pred, models, evals = test_lgbm(lightgbm, X_train, y_train, X_test, skfolds, [], params)

ValueError: DataFrame.dtypes for data must be int, float or bool. Did not expect the data types in fields TIPO_BECA

In [34]:
thold = 0.5
y_test_f1 = []
for val in y_test:
    if(val < thold): y_test_f1.append(0)
    else:
        y_test_f1.append(1)

NameError: name 'y_test' is not defined

In [ ]:
sub_df = pd.DataFrame()
sub_df['COD_ALUMNO'] = test['COD_ALUMNO']
sub_df['Morosidad'] = y_test_f1
sub_df.to_csv('sub_k17.csv', index=False)